### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.


In [1]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

In [2]:
# !pip install googletrans==4.0.0-rc1
# !pip install webdriver-manager
# !pip install requests

## 에어비앤비 주요 지역 탐색 

#### 1. 영어로된 일본 지역 리스트 본어로 전처리 
- 오시마랜드는 영오도 지원되나 일본어로 검색하는게 정확도가 높다 판단하여 일본어로 변경해준다 
- googletrans를 이용해 번역할 경우 정확도가 떨어진다
- 45개 지역 하드코딩으로 번역

In [3]:
from googletrans import Translator

main_district = pd.read_csv('../jieun/main_district_df2.csv')
# district_list  = main_district['neighbourhood_cleansed'].values
# translator = Translator()
# jp_district_list = list(map(lambda district: translator.translate(district, src='en', dest='ja').text.replace(' ',''),district_list))
# jp_district_list[: 10]


In [4]:
main_district

,neighbourhood_cleansed,count
0,Shinjuku Ku,2278
1,Taito Ku,1597
2,Sumida Ku,1290
3,Toshima Ku,1002
4,Shibuya Ku,660
5,Minato Ku,409
6,Setagaya Ku,394
7,Ota Ku,360
8,Nakano Ku,316
9,Chuo Ku,290


#### Inside AIrbne 주요 지역

In [5]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

#### 2. geojosn 파일 이용
InsideAirbnb 제공 geo 파일 사용

In [6]:
import json
import pandas as pd

In [7]:
with open('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_Airbnb\\data\\neighbourhoods.geojson') as f:
    json_f = json.loads(f.read()) # json 라이브러리 이용

geo_df = pd.DataFrame(json_f)
geo_df = pd.read_json('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_Airbnb\\data\\neighbourhoods.geojson') # pd.read_json 이용


In [8]:
geojson_path ='C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_Airbnb\\data\\neighbourhoods.geojson'
# gdf = gpd.read_file(geojson_path)
# gdf

with open(geojson_path) as f:
    geo_data = json.load(f)

geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[139.857803, 35.635799],
       [139.855301, 35.636868],
       [139.850296, 35.637665],
       [139.849899, 35.639267],
       [139.850494, 35.639999],
       [139.858398, 35.638134],
       [139.857895, 35.635864],
       [139.857803, 35.635799]]],
     [[[139.889999, 35.750801],
       [139.890396, 35.750401],
       [139.899796, 35.743732],
       [139.900101, 35.7356],
       [139.897095, 35.726665],
       [139.901398, 35.721466],
       [139.912109, 35.711266],
       [139.916107, 35.707733],
       [139.918701, 35.6978],
       [139.905899, 35.683731],
       [139.889099, 35.680599],
       [139.885803, 35.6768],
       [139.885895, 35.671131],
       [139.886597, 35.669868],
       [139.886993, 35.653866],
       [139.882095, 35.644001],
       [139.875504, 35.640533],
       [139.872894, 35.638535],
       [139.871796, 35.639332],
       [139.863495, 3

In [9]:
features = geo_data['features']

geo_dict = {}
for feature in features:
    coordinates = feature["geometry"]['coordinates']
    neighbourhood = feature["properties"]['neighbourhood']
    if neighbourhood in list(main_district['neighbourhood_cleansed'].values):
          lat_lng = coordinates[0][len(coordinates[0])//2]
          geo_dict[neighbourhood] = lat_lng

geo_dict

{'Edogawa Ku': [[139.857803, 35.635799],
  [139.855301, 35.636868],
  [139.850296, 35.637665],
  [139.849899, 35.639267],
  [139.850494, 35.639999],
  [139.858398, 35.638134],
  [139.857895, 35.635864],
  [139.857803, 35.635799]],
 'Katsushika Ku': [[139.846802, 35.796532],
  [139.8564, 35.790932],
  [139.858795, 35.794464],
  [139.862701, 35.794731],
  [139.871796, 35.797066],
  [139.877396, 35.7938],
  [139.871796, 35.788334],
  [139.871796, 35.784466],
  [139.876694, 35.780865],
  [139.881302, 35.782066],
  [139.889099, 35.783733],
  [139.893799, 35.782734],
  [139.893295, 35.778198],
  [139.880493, 35.771198],
  [139.880798, 35.764732],
  [139.887802, 35.753399],
  [139.888, 35.753201],
  [139.889999, 35.750801],
  [139.880493, 35.748867],
  [139.884903, 35.746601],
  [139.879807, 35.7416],
  [139.880005, 35.7402],
  [139.877594, 35.741135],
  [139.876495, 35.737064],
  [139.874802, 35.7374],
  [139.873703, 35.736866],
  [139.871796, 35.729797],
  [139.871109, 35.729065],
  [139.87

In [10]:
dict_list = list(geo_dict.keys())[8:30]
new_dict = {key: value for key, value in geo_dict.items() if key  in dict_list}

## 크롤링 
- 크롤링 메서드 생성 

####  1. oshimaland 웹사이트 불러오기


In [11]:
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    # driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




#### 2. 부동산 관련 사건 정보 수집

In [12]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        print('fire 개수: ', len(fire_list))
        for fire in tqdm_notebook(fire_list):
            try: 
                is_enabled = fire.is_enabled()
                actions = ActionChains(driver)                 
                if is_enabled:
                    # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                    # print('click OK')
                    if fire.is_displayed():
                        # print('display OK')
                        # x_offset = 200  # x 좌표를 조정할 값
                        # y_offset = 200  # y 좌표를 조정할 값
                        # driver.execute_script("arguments[0].scrollIntoView();", fire)
                        # actions.move_to_element_with_offset(fire, x_offset, y_offset)
                        # actions.click()
                        # actions.perform()

                        # 팝업창 클릭                         
                        actions = ActionChains(driver)                 
                        actions.move_to_element(fire).click().perform()

                        # 팝업 정보 가져오기 
                        # time.sleep(5)
                        WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                        info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                        info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                        info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                        tokyo_data_dict.append({
                            'district': district,
                            'date': info_date,
                            'address': info_address,
                            'content': info_content
                        })

                        time.sleep(3)

                    else:
                        pass
                        # print('display NOT')
                else:
                     pass
                    # print('click not')

            except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                # 에러 처리
                print(f"Error occurred: {e}")

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


#### 3. 지역 검색 


#### 3-2 도쿄 지역 위도 경도 검색

In [13]:
tokyo_data_dict = []

def search_fire_coordinate(driver, geo_dict):
    # 주요 지역 위도 경도 검색 
    for key, value in tqdm_notebook(geo_dict.items()):
       
        for lat_lng in value:
            print(key)
            region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
            region_search_tab.clear()  # 검색어 clear
            lng = lat_lng[1]
            lat = lat_lng[0]
            #경도 위도 검색
            search  = f'{lng} {lat}'
            region_search_tab.send_keys(search)
            time.sleep(2)
            # 검색 버튼 클릭 
            search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
            search_btn.send_keys(Keys.ENTER)
            # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
            time.sleep(8)
            # fire 객체 정보 수집 
            get_fire(driver,tokyo_data_dict, key)



### 크롤링 

In [14]:

dict_list = list(geo_dict.keys())[8:30]
rest_dict = {key: value for key, value in geo_dict.items() if key in dict_list}
list(rest_dict.keys())

['Fussa Shi',
 'Bunkyo Ku',
 'Shinjuku Ku',
 'Adachi Ku',
 'Sumida Ku',
 'Nerima Ku',
 'Taito Ku',
 'Kodaira Shi',
 'Koganei Shi',
 'Higashimurayama Shi',
 'Hino Shi',
 'Kunitachi Shi',
 'Kokubunji Shi',
 'Itabashi Ku',
 'Setagaya Ku',
 'Ota Ku',
 'Nakano Ku',
 'Shibuya Ku',
 'Toshima Ku',
 'Chofu Shi',
 'Suginami Ku',
 'Tama Shi']

In [15]:
if __name__ == "__main__":
    # 드라이버 생성 및 웹 로딩
    driver = webdriver.Firefox()
    url = 'https://www.oshimaland.co.jp/'
    loadWeb(driver, url)
    search_fire_coordinate(driver, rest_dict)


driver.quit()

  0%|          | 0/22 [00:00<?, ?it/s]

Fussa Shi
fire 개수:  54


  0%|          | 0/54 [00:00<?, ?it/s]

Fussa Shi
fire 개수:  2


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Tama Shi
fire 개수:  2


  0%|          | 0/2 [00:00<?, ?it/s]

Tama Shi
fire 개수:  2


  0%|          | 0/2 [00:00<?, ?it/s]

#### 발생 에러들

1. WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"} : Chrome driver 문제
2. StaleElementReferenceException: Message: The element with the reference b3212e1d-3be9-4d7a-9bd2-9f736fc42e4a is stale; either its node document is not the active document, or it is no longer connected to the DOM
3. AttributeError: 'NoneType' object has no attribute 'text'
4. MoveTargetOutOfBoundsException

In [16]:
driver.quit()

### 크롤링 데이터 저장

- 230725 Fussa Shi 지역 부터 데이터 수집 필요 ( 7개의 지역만 수집했음에도 2시간 이상 소요)

```
 'Fussa Shi', 'Bunkyo Ku', 'Shinjuku Ku', 'Adachi Ku', 'Sumida Ku', 'Nerima Ku', 'Taito Ku', 'Kodaira Shi', 'Koganei Shi', 'Higashimurayama Shi', 'Hino Shi', 'Kunitachi Shi', 'Kokubunji Shi', 'Itabashi Ku', 'Setagaya Ku', 'Ota Ku', 'Nakano Ku', 'Shibuya Ku', 'Toshima Ku', 'Chofu Shi', 'Suginami Ku', 'Tama Shi', 'Akishima Shi', 'Arakawa Ku', 'Machida Shi', 'Nishitokyo Shi', 'Meguro Ku', 'Fuchu Shi', 'Kita Ku', 'Tachikawa Shi', 'Hachioji Shi', 'Mitaka Shi', 'Okutama Machi', 'Musashino Shi', 'Musashimurayama Shi', 'Ome Shi', 'Akiruno Shi', 'Hamura Shi'
```

In [17]:
tokyo_data_dict

[{'district': 'Bunkyo Ku',
  'date': '2009',
  'address': '東京都文京区湯島二丁目4-8',
  'content': 'マンションの部屋番号は忘れてしまったのですが、\n２００９年夏、相場より賃料が安い部屋があったので問い合わせたら\n自殺があったので、リフォームしたとのこと。とても綺麗な部屋になっていました。'},
 {'district': 'Bunkyo Ku',
  'date': '2009',
  'address': '東京都文京区湯島二丁目4-8',
  'content': 'マンションの部屋番号は忘れてしまったのですが、\n２００９年夏、相場より賃料が安い部屋があったので問い合わせたら\n自殺があったので、リフォームしたとのこと。とても綺麗な部屋になっていました。'},
 {'district': 'Bunkyo Ku',
  'date': '10年位前',
  'address': '東京都文京区本郷三丁目4-6',
  'content': 'ビル裏手の駐車場に屋上からの飛び降り。近隣住民の親族で普段は遠方にいた人らしい。ビルとは無関係'},
 {'district': 'Bunkyo Ku',
  'date': '令和3年3月27日',
  'address': '東京都文京区湯島三丁目3-2',
  'content': '被疑事実：傷害致死→不起訴\n死因：敗血症'},
 {'district': 'Bunkyo Ku',
  'date': '2013',
  'address': '東京都文京区本郷三丁目22-2 フローレンスパレス御茶ノ水フィアンコ ６０１',
  'content': 'ミイラ（白骨化するまで発見されず）'},
 {'district': 'Bunkyo Ku',
  'date': '2005年頃',
  'address': '東京都文京区湯島一丁目7-5',
  'content': '東京ガーデンパレス4階客室\nドアノブに紐をかけ教師が首吊り自殺'},
 {'district': 'Bunkyo Ku',
  'date': '2005年頃',
  'address': '東京都文京区湯島一丁目7-5',
  'content': '東京ガ

In [27]:
len(tokyo_data_dict)

3619

In [30]:
tokyo_df = pd.DataFrame(tokyo_data_dict)
# tokyo_df = tokyo_df[tokyo_df['district']!='Kita Ku']
tokyo_df

,district,date,address,content
0,Bunkyo Ku,2009,東京都文京区湯島二丁目4-8,マンションの部屋番号は忘れてしまったのですが、\n２００９年夏、相場より賃料が安い部屋があっ...
1,Bunkyo Ku,2009,東京都文京区湯島二丁目4-8,マンションの部屋番号は忘れてしまったのですが、\n２００９年夏、相場より賃料が安い部屋があっ...
2,Bunkyo Ku,10年位前,東京都文京区本郷三丁目4-6,ビル裏手の駐車場に屋上からの飛び降り。近隣住民の親族で普段は遠方にいた人らしい。ビルとは無関係
3,Bunkyo Ku,令和3年3月27日,東京都文京区湯島三丁目3-2,被疑事実：傷害致死→不起訴\n死因：敗血症
4,Bunkyo Ku,2013,東京都文京区本郷三丁目22-2 フローレンスパレス御茶ノ水フィアンコ ６０１,ミイラ（白骨化するまで発見されず）
...,...,...,...,...
3614,Tama Shi,,,
3615,Tama Shi,,,
3616,Tama Shi,,,
3617,Tama Shi,,,


In [29]:
# 결측치 제거 
# df_cleaned = tokyo_df.dropna(how='any', axis=0)

# for column in df_cleaned.columns:
#     df_cleaned = df_cleaned[df_cleaned[column] != '']

# # 중복데이터 제거 
# df_cleaned = df_cleaned.drop_duplicates()
# df_cleaned

In [20]:
# df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 31
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  10 non-null     object
 1   date      10 non-null     object
 2   address   10 non-null     object
 3   content   10 non-null     object
dtypes: object(4)
memory usage: 400.0+ bytes


In [31]:
# tokyo_df.to_excel('selenium_tokyo_data_geo.xlsx')
tokyo_df.to_csv('selenium_tokyo_data_geo_j.csv')

In [24]:
# df = pd.read_csv('selenium_tokyo_data_geo.csv', index_col=0)

# df

,district,date,address,content
0,Bunkyo Ku,2009,東京都文京区湯島二丁目4-8,マンションの部屋番号は忘れてしまったのですが、\n２００９年夏、相場より賃料が安い部屋があっ...
2,Bunkyo Ku,10年位前,東京都文京区本郷三丁目4-6,ビル裏手の駐車場に屋上からの飛び降り。近隣住民の親族で普段は遠方にいた人らしい。ビルとは無関係
3,Bunkyo Ku,令和3年3月27日,東京都文京区湯島三丁目3-2,被疑事実：傷害致死→不起訴\n死因：敗血症
4,Bunkyo Ku,2013,東京都文京区本郷三丁目22-2 フローレンスパレス御茶ノ水フィアンコ ６０１,ミイラ（白骨化するまで発見されず）
5,Bunkyo Ku,2005年頃,東京都文京区湯島一丁目7-5,東京ガーデンパレス4階客室\nドアノブに紐をかけ教師が首吊り自殺
7,Bunkyo Ku,平成31年4月20日,東京都文京区本郷三丁目2-15新興ビル1階,火災による死亡
8,Bunkyo Ku,2018年10月頃,東京都文京区本郷三丁目3-15,順天堂の患者が不法侵入の後\n７階の手擦りから飛び降り自殺、\nビルから出たら凄い音がして死...
19,Bunkyo Ku,2008年頃,東京都千代田区神田駿河台二丁目3−1,ロッテリアの入るビル8階から飛び降り、一階五右衛門パスタに降りる階段手すりに激突、通行人多数...
20,Bunkyo Ku,2017年8月以前,東京都文京区本郷二丁目15-1,クレール本郷 2階 所有者の母が室内で孤高死。死後3日に発見。
31,Bunkyo Ku,ここ半年ほど,東京都千代田区神田淡路町二丁目101,削除されているけど、2811号室、ここ半年くらいやたら警察官が出入りしてる。\n傷害・脅迫が...


#### 기존 오시마 데이터랑 중복 확인 

In [ ]:
oshima_yun = pd.read_csv('../../Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col=0)
oshima_ji = pd.read_csv('../../Oshimaland_data/jieun/oshimaland_dataset_final.csv')


In [ ]:
# 결측치 제거 
oshima_yun = oshima_yun.dropna(how='any', axis=0)

for column in oshima_yun.columns:
    oshima_yun = oshima_yun[oshima_yun[column] != '']


# # 중복데이터 제거 
oshima_yun = oshima_yun.drop_duplicates()
oshima_yun = oshima_yun.reindex(columns=['district', 'address', 'content', 'date'])
oshima_yun.rename(columns={'content': 'info'}, inplace=True)
oshima_yun

In [ ]:
oshima_ji_rename = oshima_ji.reindex(columns=[ 'address', 'info', 'occurred_date'])
oshima_ji_rename.rename(columns={'occurred_date': 'date'}, inplace=True)
oshima_ji_rename = oshima_ji_rename.drop_duplicates()
oshima_ji_rename.reset_index(drop=True, inplace=True)
oshima_ji_rename.dropna(inplace=True)
oshima_ji_rename = oshima_ji_rename.reindex(columns = ['district', 'address', 'info', 'date'])
oshima_ji_rename['district'] = oshima_ji_rename['address'].apply(lambda x: next((district for district in jp_district_list if district in x), x))
oshima_ji_rename.reindex(columns=['district', 'address', 'info', 'occurred_date'])

oshima_ji_rename


In [ ]:
combined_df = pd.concat([oshima_yun, oshima_ji_rename], ignore_index=True)
combined_df = combined_df.drop_duplicates()
combined_df

In [ ]:
new_oshima = df.reindex(columns=['district', 'address', 'content', 'date'])
new_oshima.rename(columns={'content': 'info'}, inplace=True)
new_oshima

In [ ]:
combined_df

In [ ]:
combined_df2 = pd.concat([combined_df, new_oshima], ignore_index=True)
combined_df2 = combined_df2.drop_duplicates()
combined_df2

#### 중복 데이터 확인 
- 7개의 지역만 추가 트롤링 했을때 기존 807개의 데이터에서  1183개의 데이터 즉 376 개의 새로운 데이터 수집 
- 대략적으로 46개의 지역을 다 크롤링 할 시 2000개 이상의 데이터 추가 수집 가능 

In [ ]:
# address 컬럼의 값이 중복되면 하나만 남기고 제거
combined_df2 = combined_df2.drop_duplicates(subset=['address'], keep='first')
combined_df2

#### 지도 내 줌인 줌아웃 실패 

In [ ]:
# from selenium.webdriver.common.action_chains import ActionChains
# # https://stackoverflow.com/questions/64474653/how-to-webscrape-a-canvas-element-with-python-selenium
# canvas = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > canvas.rt-root')
# action = ActionChains(driver)

# map_canvas = driver.find_element_by_css_selector("div.mk-map-view > canvas.rt-root")

# from selenium.webdriver.common.action_chains import ActionChains

# # 지도 요소 위에서 마우스를 움직임
# actions = ActionChains(driver)
# actions.move_to_element(map_canvas).click().perform()

# actions.move_by_offset(0, 200).perform() # 줌인만 가능 

# #줌인 줌아웃 반복 
# action.move_to_element(canvas).move_by_offset(0, 0).click_and_hold().perform()
# action.move_by_offset(-200,0).release().perform()

#### Request API 이용한 방법 

In [ ]:
import json
from  bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib.request as req
from tqdm import tqdm, tqdm_notebook
import time
import requests

In [ ]:
#TODO shinjuku ku test
search_region = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
search_region.send_keys(jp_district_list[20])
time.sleep(2)
# 검색 버튼 클릭 
search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
search_btn.send_keys(Keys.ENTER)

time.sleep(2)

In [ ]:
json_url = 'https://api.oshimaland.co.jp/map'
headers = {
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://www.oshimaland.co.jp"
}
data = {
    # 'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
    'keys' : ['1330021123013021',  '1330021123013023',  '1330021123013201']
}

response = requests.post(json_url, headers=headers, json=data)
json_data = response.json()

In [ ]:
from itertools import chain

origin = list(data.values())
flatten_list = list(chain(*origin))
flatten_list

In [ ]:
jsonValueList = []
for idx in range(len(flatten_list)):
    for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
        jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])

In [ ]:
getDataList2 = []
for idx in tqdm_notebook(range(len(jsonValueList))):
    json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
    url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
    code = req.urlopen(url)
    soup = BeautifulSoup(code, 'html.parser')
    json_data = json.loads(soup.text)
    getDataList2.append({
        'info' : json_data['info'],
        'address' : json_data['ad'],
        'dt' : json_data['dt'],
        'cr' : json_data['cr'],
     })